## Day 47 Lecture 2 Assignment

In this assignment, we will perform K-Medoids clustering using responses to a survey about student life at a university.

In [1]:
# !pip install pyclustering

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from pyclustering.cluster.kmedoids import kmedoids
import random

This dataset consists of 35 binary features, each corresponding to a yes/no question that characterizes the student taking the survey.

This dataset contains a large number of features, each corresponding to a survey question. The feature name summarizes the survey question, so we will not list them all out here.

Load the dataset.

In [3]:
# load dataset
data = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/student_life_survey.csv')


For our analysis, we will focus on a specific subset of the survey that is focused on stress. These questions all begin with the string 'Q5'. Filter the columns that meet this criteria (should be 10 in total).

In [4]:
# select only columns for Q5
data = data.loc[:, data.columns.str.contains('Q5')].reset_index(drop=True)

The pyclustering implementation of kmedoids supports a variety of distance metrics, but they are primarily for numeric data. We will be using SMC/Hamming dissimilarity and precomputing the similarity matrix (an alternative would be to specify a user-defined function, which you are welcome to try in addition).

We'll assume for the next step that a pair of negative values (i.e. both responses are "no") is as informative as a pair of positive values. Compute the full distance/dissimilarity matrix for the survey data using matching/hamming distance.

In [5]:
# dissimilarity matrix with hamming distance
dist = pdist(data, metric='hamming')
dist_df = pd.DataFrame(squareform(dist), index=data.index, columns=data.index)

In [6]:
# Creating df with pair differences for use later
dist_df['id'] = dist_df.index
sims = dist_df.melt(id_vars='id')

Using the dissimilarity matrix, perform kmedoids clustering using k=2. Set the initial medoids randomly. Note that pyclustering expects the distance matrix to be a numpy array; a pandas dataframe may cause errors. 

Which survey responses are chosen as the cluster representatives? Print out the details of these responses.

In [94]:
# # answer goes here
initial_medoids = dist_df.sample(2).index
kmedoids_instance = kmedoids(dist_df, initial_medoids, data_type='distance_matrix')
kmedoids_instance.process()

clusters = kmedoids_instance.get_clusters()
medoids = kmedoids_instance.get_medoids()
for i in range(len(clusters)):
  dist_df.loc[clusters[i], 'cluster']=i


print('The initial medoids had indeces of', 
      initial_medoids.values[0], 'and', initial_medoids.values[1])
dist_df.loc[medoids].style.background_gradient()

The initial medoids had indeces of 2384 and 1997


If you run the previous cell a few times, you'll probably notice that the medoids are very sensitive to initialization. A common approach to produce well-separated clusters is to choose initial medoids that are far apart. Re-run the previous process, except with a random pair of medoids that have a dissimilarity of 0.8 or higher. Are the results more stable now? How would you describe the typical clusters you see?

In [19]:
# selecting only pairs with differences of .8 or higher
sims = sims.loc[sims['value'] >= .8]


In [96]:
sample = sims.sample(1)
initial_medoids = [sample.iloc[0, 0], sample.iloc[0, 1]]


kmedoids_instance = kmedoids(dist_df, initial_medoids, data_type='distance_matrix')
kmedoids_instance.process()

clusters = kmedoids_instance.get_clusters()
medoids = kmedoids_instance.get_medoids()
for i in range(len(clusters)):
  dist_df.loc[clusters[i], 'cluster']=i


print('The initial medoids had indeces of', 
      initial_medoids[0], 'and', initial_medoids[1])
dist_df.loc[medoids].style.background_gradient()

The initial medoids had indeces of 1452 and 2416


Even after selecting a pair with dissimilarity above .8, the clusters still varied signicantly with initialization. Because of this it's hard to draw any conclusions about the clusters.